In [17]:
import pandas as pd
import numpy as np
import re
import jieba
from tqdm import tqdm

In [18]:
brand_df = pd.read_csv('filtered_brands_df.csv', header=None, index_col=None)
brand_df.columns=['name','id']
end_words = pd.read_csv('userdict_endword.txt',sep=' ', header = None)[0].tolist()

for list_num in tqdm(range(14)):
    try:
        uid_list = pd.read_csv('final_uid_list/final_uid_list'+str(list_num)+'.csv',index_col=0,header=None)
        info_path = 'product_info/'+ str(list_num)+'/'

        info_sum = pd.DataFrame(columns = ['url', 'bid', 'name', 'tmallname', 'brand', 'price', 'popularity', 
                                       'score', 'posi', 'neg','uid'])

        for uid in list(uid_list.iloc[:,0]):
            product_info = pd.read_csv(info_path+str(uid)+'_prodinfo.csv',index_col=0)
            if len(product_info) != 0:
                product_info['uid'] = uid
                info_sum = pd.concat([info_sum,product_info], ignore_index=True, sort=False)
        length = len(info_sum)
        del_lst = []

        for info_idx in range(length):
            t_name = info_sum['brand'][info_idx]
            tm_name = info_sum['tmallname'][info_idx]
            t_name = t_name.upper()
            t_name = re.sub(r'[^a-zA-Z0-9\u4e00-\u9fff]' , '', t_name)
            uid = info_sum['uid'][info_idx]
            bid = info_sum['bid'][info_idx]
            post_info = pd.read_csv('product_name/'+ str(list_num)+'/'+str(uid)+'_prodname.csv',index_col=None)
            brand_id = post_info['品牌'][post_info[post_info['bid'] == bid].index.tolist()[0]]
            name_lst = brand_df['name'][brand_df['id'] == brand_id].tolist()
            result = 0
            for name in name_lst:
                name = re.sub(r'[^a-zA-Z0-9\u4e00-\u9fff]' , '', name)
                if all(elem in list(t_name) for elem in list(name)):
                    result = 1
                    break
            if not any(elem in tm_name for elem in end_words):
                result = 0
            if result == 0:
                del_lst.append(info_idx)
                
        info_sum = info_sum.drop(del_lst)
        info_sum.to_csv('cleaned_product_info/result_uid'+str(list_num)+'.csv', encoding = "utf_8_sig")
        
    except Exception as e:
        print(e)

100%|██████████| 14/14 [01:57<00:00,  8.39s/it]
